# Importações



In [4]:
# Carregar e visualizar os dados
import pandas as pd
import numpy as np

#Biblioteca para gerar e manipular as word embeddings
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

#Biblioteca para preprocessar os textos 
import spacy

#O tqdm é utilizado para exibir uma barra de progresso
#Os demais comandos são utilizados para o correto funcionamento no Colab
from tqdm import tqdm
from IPython import get_ipython
def tqdm_clear(*args, **kwargs):
    getattr(tqdm, '_instances', {}).clear()

#Importando bibliotecas do ScikitLearn para: 
# - Faze a divisão entre treino e teste
from sklearn.model_selection import train_test_split
# - Importanto um modelo de classificação SVC (Support Vector Classifier)
from sklearn.svm import SVC
# - Importante a função para avaliar o resultado da classificação
from sklearn.metrics import f1_score

# Carregando o Dataset

In [2]:
df = pd.read_csv('/content/drive/My Drive/Datasets/Texts/CSTR.csv')

In [3]:
df

,file_name,text,class
0,126.txt,Rhetorical (Rhet) is a programming / knowledge...,ArtificiallIntelligence
1,5.txt,Reduction is the operation of transforming a p...,ArtificiallIntelligence
2,48.txt,"For years, researchers have used knowledge-int...",ArtificiallIntelligence
3,81.txt,Proceedings of a workshop held in conjunction ...,ArtificiallIntelligence
4,25.txt,The Medication Advisor is the latest project o...,ArtificiallIntelligence
...,...,...,...
294,39.txt,Scoring protocols are a broad class of voting ...,Theory
295,5.txt,We study the behavior of Range Voting and Norm...,Theory
296,28.txt,Using entropy of traffic distributions has bee...,Theory
297,27.txt,We study the complexity of influencing electio...,Theory


In [46]:
#Dividindo a coleção entre treino e teste
df_treino, df_teste = train_test_split(df, test_size=0.3, stratify=df['class'])

# Pré-processamento dos Textos

In [47]:
# Initializando o spacy, porém, desabilitando recursos que não iremos utilizar
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

In [48]:
# Preprocessamento dos textos
def preprocessamento (texto): 
  final_tokens = []
  doc = nlp(texto)
  for token in doc:
    if(token.is_alpha and not token.is_stop):
          final_tokens.append(token.lemma_.lower())
      
  return final_tokens

In [55]:
# Retornando um objeto TaggedDocument 
# O objeto TaggedDocument contém os tokens (words) e um identificador para os documentos (tags)
def tagged_data(texts): 
  return [TaggedDocument(words=preprocessamento(text), tags=[str(i)]) for i, text in enumerate(texts)]

In [167]:
# Construindo os vetores de treino
# Utiliza-se o model.docvecs[(string da tag)] para retornar um vetor de um documento
def constuir_vetores_treino(model, tagged_data, num_dim):
  matrix = np.zeros((len(tagged_data),num_dim))
  for i in range(len(tagged_data)):
    matrix[i] = model.docvecs[tagged_data[i][1][0]]
  return matrix 

In [168]:
# Construindo os vetores dos novos documentos (teste)
# Utiliza-se a função infer_vector() e passasse uma lista com as palavras de um documento
def constuir_vetores_teste(model, tagged_data, num_dim):
  matrix = np.zeros((len(tagged_data),num_dim))
  for i in range(len(tagged_data)):
    matrix[i] = model.infer_vector(tagged_data[i][0])
  return matrix 

#Gerando a Representação do Conjunto de Treino

In [169]:
#Definindo o numero de dimesões
num_dim = 100

In [170]:
#TaggedDocument é a estrutura necessária para o treinamento do modelo Doc2Vec
tagged_treino = tagged_data(df_treino['text'])

In [171]:
tagged_treino

[TaggedDocument(words=['order', 'understand', 'natural', 'language', 'necessary', 'understand', 'intention', 'utterance', 'speak', 'model', 'dialogue', 'collaboration', 'agent', 'communicative', 'intention', 'see', 'agent', 'try', 'affect', 'collaboration', 'previous', 'work', 'intention', 'recognition', 'approach', 'dialogue', 'focus', 'small', 'subset', 'agent', 'collaboration', 'paradigm', 'master', 'slave', 'unable', 'account', 'dialogue', 'paradigm', 'mix', 'initiative', 'collaboration', 'previous', 'work', 'model', 'dialogue', 'agent', 'plan', 'dialogue', 'agent', 'act', 'restrict', 'dialogue', 'model', 'coverage', 'case', 'model', 'dialogue', 'collaboration', 'act', 'planning', 'occur', 'paper', 'present', 'collaborative', 'problem', 'solve', 'model', 'dialogue', 'model', 'able', 'account', 'wide', 'array', 'dialogue', 'previous', 'model', 'cover', 'cover', 'spectrum', 'collaboration', 'paradigm', 'master', 'slave', 'mix', 'initiative', 'dialogue', 'interleave', 'acting', 'plann

In [172]:
#Vector size: número de dimensões da embedding
#min_count: número mínimo de vezes que uma palavra deve aparecer na coleção para ser considerada no vocabulário
#dm: se 1 significa que será utilizado o modelo DM, e 0 significa a utilização do modelo DBOW
#dm_contate: 1 significa a concatenação dos vetores e 0 significa a média
model = Doc2Vec(vector_size=100, min_count=2, dm=1, dm_concat = 1,epochs=100)
model.build_vocab(tagged_treino)
model.train(tagged_treino,total_examples=model.corpus_count,epochs=model.iter)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  import sys


In [173]:
#Reduzindo o uso de memória
model.delete_temporary_training_data(keep_doctags_vectors=True, keep_inference=True)

In [174]:
#Retornando o vetor de um documento específico
model.docvecs['99']

array([-0.21350163,  0.518985  ,  0.2621769 , -0.50123894,  0.30604535,
       -0.76276064, -0.03692596, -0.10289425, -0.22454019,  0.3223731 ,
        0.21365316, -0.25018468,  0.03813893, -0.3744234 , -0.25857016,
        0.31966877, -0.41107163,  0.16822456,  0.17201322,  0.20553581,
       -0.1269353 , -0.4818998 ,  0.07511721,  0.3597196 , -0.33525798,
        0.07259433, -0.03015873, -0.2753913 ,  0.627168  , -0.20994936,
        0.30568287,  0.00772855, -0.41094124, -0.08683598,  0.03580499,
        0.08384772,  0.3727823 ,  0.33311275, -0.02928626, -0.00911088,
       -0.1693627 ,  0.16375813,  0.15085629,  0.4208998 , -0.5075453 ,
       -0.1571005 , -0.11056541, -0.19484924,  0.4380839 ,  0.10833288,
       -0.2776975 , -0.36709553, -0.04924485, -0.25200897,  0.07802427,
       -0.46872866, -0.5469607 ,  0.05443887, -0.14133418, -0.13292979,
        0.00105906,  0.01498361, -0.44271678, -0.12671605, -0.82263017,
       -0.1720718 ,  0.11835211, -0.30545503, -0.11229146, -0.26

In [141]:
len(tagged_treino)

209

In [175]:
#Construindo a representação para o conjunto de treino
repr_treino = constuir_vetores_treino(model, tagged_treino, num_dim)

In [176]:
repr_treino

array([[-0.30389971,  0.06507325,  0.22577207, ..., -0.22264957,
        -0.62324685,  0.24069855],
       [-0.20261821, -0.25733793,  0.32939449, ..., -0.03577409,
         0.04425891,  0.0586013 ],
       [ 0.46279982,  0.22318052, -0.02561086, ...,  0.19592594,
         0.46459264,  0.2526679 ],
       ...,
       [ 0.08866474,  0.09879918, -0.33386961, ..., -0.00601573,
         0.17756955, -0.39717343],
       [-0.26908183,  1.23868537,  0.09164279, ...,  0.53623676,
         0.55227411, -0.07785906],
       [ 0.26481882, -0.41381058, -0.06508511, ..., -0.07047043,
        -0.13571241,  0.24601074]])

In [177]:
repr_treino.shape

(209, 100)

# Construindo um modelo de Classificação no Treino

In [201]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=7, metric='cosine')

In [202]:
classifier.fit(repr_treino, df_treino['class'])

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='cosine',
                     metric_params=None, n_jobs=None, n_neighbors=7, p=2,
                     weights='uniform')

# Avaliando os resultados da classificação no conjunto de teste

## Construindo a representação do conjunto de teste

In [203]:
tagged_teste = tagged_data(df_teste['text'])

In [204]:
repr_teste = constuir_vetores_teste(model, tagged_teste, num_dim)

In [205]:
repr_teste

array([[-0.0185711 ,  0.20279531, -0.42402735, ..., -1.3776629 ,
        -0.67177463, -0.75650001],
       [-0.16915292,  0.28923172, -0.48771727, ..., -0.65883189,
        -0.92237693, -0.67874187],
       [ 0.52248931, -0.0345635 , -0.00295892, ..., -0.98199719,
        -0.61600173, -0.69399577],
       ...,
       [ 1.19139814, -1.7691282 , -0.9763096 , ..., -0.95323598,
        -0.41137624,  0.44832253],
       [ 0.47533625, -0.44106215, -1.22815061, ..., -1.88681746,
        -2.02031088, -0.17406216],
       [-0.69027042, -0.49125186,  0.96908212, ..., -1.43490148,
        -0.18420385,  2.00309992]])

In [206]:
repr_teste.shape

(90, 100)

## Predizendo a classe dos exemplos de teste

In [207]:
classifier.predict(repr_teste)

array(['ArtificiallIntelligence', 'ArtificiallIntelligence',
       'ArtificiallIntelligence', 'Robotics', 'ArtificiallIntelligence',
       'ArtificiallIntelligence', 'ArtificiallIntelligence', 'Robotics',
       'Robotics', 'ArtificiallIntelligence', 'Robotics', 'Theory',
       'Robotics', 'ArtificiallIntelligence', 'Theory', 'Robotics',
       'ArtificiallIntelligence', 'Robotics', 'Robotics', 'Theory',
       'Robotics', 'ArtificiallIntelligence', 'Theory', 'Robotics',
       'Robotics', 'Theory', 'Robotics', 'Theory',
       'ArtificiallIntelligence', 'ArtificiallIntelligence', 'Theory',
       'ArtificiallIntelligence', 'Robotics', 'ArtificiallIntelligence',
       'Robotics', 'Systems', 'Theory', 'ArtificiallIntelligence',
       'Systems', 'Robotics', 'ArtificiallIntelligence',
       'ArtificiallIntelligence', 'ArtificiallIntelligence', 'Robotics',
       'Robotics', 'Robotics', 'Theory', 'ArtificiallIntelligence',
       'Robotics', 'ArtificiallIntelligence', 'ArtificiallInt

#Avaliando a performance da predição dos exemplos de teste

In [208]:
f1_score(df_teste['class'], classifier.predict(repr_teste), average='macro')

0.7838974031502767

In [209]:
f1_score(df_teste['class'], classifier.predict(repr_teste), average='micro')

0.7444444444444445

# Comparando com a BOW

In [210]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [214]:
def preprocessamento2 (texto): 
  final_tokens = []
  doc = nlp(texto)
  for token in doc:
    if(token.is_alpha and not token.is_stop):
          final_tokens.append(token.lemma_.lower())
      
  return ' '.join(final_tokens)

In [211]:
vetorizador = TfidfVectorizer()

In [215]:
bow_treino = vetorizador.fit_transform(df_treino['text'].apply(preprocessamento2))

In [216]:
classifier.fit(bow_treino, df_treino['class'])

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='cosine',
                     metric_params=None, n_jobs=None, n_neighbors=7, p=2,
                     weights='uniform')

In [217]:
repr_teste = vetorizador.transform(df_teste['class'].apply(preprocessamento2))
f1_score(df_teste['class'], classifier.predict(repr_teste), average='macro')

0.446969696969697